## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-22-14-49-55 +0000,nyt,"Frank Caprio, Kind Judge on Rhode Island TV, D...",https://www.nytimes.com/2025/08/21/us/judge-fr...
1,2025-08-22-14-49-42 +0000,nyt,Has Russia Advanced? Ukrainians Use This Onlin...,https://www.nytimes.com/2025/08/22/world/europ...
2,2025-08-22-14-49-28 +0000,nyt,Was Head-Binding an Ancient Way to Get Ahead?,https://www.nytimes.com/2025/08/22/science/ant...
3,2025-08-22-14-47-57 +0000,bbc,Revolution Beauty to 'reset' after sale falls ...,https://www.bbc.com/news/articles/c36j2nxg202o...
4,2025-08-22-14-47-16 +0000,bbc,Global postal services halt US deliveries over...,https://www.bbc.com/news/articles/cx2p17xypgko...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2369/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
51,trump,60
194,new,22
454,out,15
179,menendez,12
231,gaza,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
258,2025-08-21-20-32-00 +0000,wsj,N.Y. Appeals Court Throws Out $500 Million Civ...,https://www.wsj.com/us-news/law/ny-appeals-cou...,115
66,2025-08-22-11-36-52 +0000,nypost,Ukraine obliterates critical part of Russia’s ...,https://nypost.com/2025/08/22/world-news/ukrai...,115
216,2025-08-21-23-19-00 +0000,wsj,"Alina Habba, Trump’s Top Prosecutor in New Jer...",https://www.wsj.com/us-news/law/alina-habba-ne...,114
274,2025-08-21-19-17-34 +0000,nyt,NY Appeals Court Throws Out Trump’s Half-Billi...,https://www.nytimes.com/2025/08/21/nyregion/tr...,114
264,2025-08-21-20-11-18 +0000,nypost,Trump pick Alina Habba is unlawfully serving a...,https://nypost.com/2025/08/21/us-news/trump-pi...,113


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
258,115,2025-08-21-20-32-00 +0000,wsj,N.Y. Appeals Court Throws Out $500 Million Civ...,https://www.wsj.com/us-news/law/ny-appeals-cou...
211,71,2025-08-21-23-43-37 +0000,nypost,California Gov. Gavin Newsom signs redistricti...,https://nypost.com/2025/08/21/us-news/californ...
156,60,2025-08-22-04-36-24 +0000,wapo,"Erik Menendez, who was convicted of killing hi...",https://www.washingtonpost.com/nation/2025/08/...
17,57,2025-08-22-14-25-00 +0000,wsj,The Federal Reserve said it would no longer in...,https://www.wsj.com/economy/central-banking/fe...
13,57,2025-08-22-14-29-02 +0000,nyt,Jerome Powell Suggests Fed Will Soon Cut Inter...,https://www.nytimes.com/2025/08/22/business/po...
142,55,2025-08-22-06-30-55 +0000,nypost,Florida nurse Alexis Von Yates takes plea deal...,https://nypost.com/2025/08/22/us-news/florida-...
66,55,2025-08-22-11-36-52 +0000,nypost,Ukraine obliterates critical part of Russia’s ...,https://nypost.com/2025/08/22/world-news/ukrai...
218,52,2025-08-21-23-02-09 +0000,nypost,Israeli stats show civilian death rate in Gaza...,https://nypost.com/2025/08/21/world-news/israe...
335,48,2025-08-21-16-19-40 +0000,nypost,Ex-Biden White House spokesman says he only sa...,https://nypost.com/2025/08/21/us-news/ex-biden...
20,46,2025-08-22-14-24-00 +0000,wsj,The FBI on Friday searched the Maryland home o...,https://www.wsj.com/politics/policy/fbi-raids-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
